In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
ss = SparkSession.builder.getOrCreate()
sc = ss.sparkContext

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/24 00:34:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Create Data Frame

In [2]:
#Load the data and create an RDD (16 pixels and label)
pen_raw = sc.textFile("../Data/penbased.dat", 4)\
            .map(lambda x:  x.split(", "))\
            .map(lambda row: [float(x) for x in row])
pen_raw.take(1)

[[47.0,
  100.0,
  27.0,
  81.0,
  57.0,
  37.0,
  26.0,
  0.0,
  0.0,
  23.0,
  56.0,
  53.0,
  100.0,
  90.0,
  40.0,
  98.0,
  8.0]]

In [3]:
#Create a DataFrame
from pyspark.sql.types import *

penschema = StructType([
    StructField("pix1",DoubleType(),True),
    StructField("pix2",DoubleType(),True),
    StructField("pix3",DoubleType(),True),
    StructField("pix4",DoubleType(),True),
    StructField("pix5",DoubleType(),True),
    StructField("pix6",DoubleType(),True),
    StructField("pix7",DoubleType(),True),
    StructField("pix8",DoubleType(),True),
    StructField("pix9",DoubleType(),True),
    StructField("pix10",DoubleType(),True),
    StructField("pix11",DoubleType(),True),
    StructField("pix12",DoubleType(),True),
    StructField("pix13",DoubleType(),True),
    StructField("pix14",DoubleType(),True),
    StructField("pix15",DoubleType(),True),
    StructField("pix16",DoubleType(),True),
    StructField("label",DoubleType(),True)
])

dfpen = ss.createDataFrame(pen_raw, penschema)

In [4]:
dfpen.show()

+-----+-----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
| pix1| pix2|pix3| pix4| pix5| pix6| pix7| pix8| pix9|pix10|pix11|pix12|pix13|pix14|pix15|pix16|label|
+-----+-----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
| 47.0|100.0|27.0| 81.0| 57.0| 37.0| 26.0|  0.0|  0.0| 23.0| 56.0| 53.0|100.0| 90.0| 40.0| 98.0|  8.0|
|  0.0| 89.0|27.0|100.0| 42.0| 75.0| 29.0| 45.0| 15.0| 15.0| 37.0|  0.0| 69.0|  2.0|100.0|  6.0|  2.0|
|  0.0| 57.0|31.0| 68.0| 72.0| 90.0|100.0|100.0| 76.0| 75.0| 50.0| 51.0| 28.0| 25.0| 16.0|  0.0|  1.0|
|  0.0|100.0| 7.0| 92.0|  5.0| 68.0| 19.0| 45.0| 86.0| 34.0|100.0| 45.0| 74.0| 23.0| 67.0|  0.0|  4.0|
|  0.0| 67.0|49.0| 83.0|100.0|100.0| 81.0| 80.0| 60.0| 60.0| 40.0| 40.0| 33.0| 20.0| 47.0|  0.0|  1.0|
|100.0|100.0|88.0| 99.0| 49.0| 74.0| 17.0| 47.0|  0.0| 16.0| 37.0|  0.0| 73.0| 16.0| 20.0| 20.0|  6.0|
|  0.0|100.0| 3.0| 72.0| 26.0| 35.0| 85.0| 35.0|100.0| 71.0| 73.0| 97.0| 

## Create a data frame includes "feature" and "label"

In [5]:
# Merging the data with Vector Assembler.
from pyspark.ml.feature import VectorAssembler
va = VectorAssembler(outputCol="features", 
                     inputCols=dfpen.columns[0:-1]) #except the last col.
penlpoints = va.transform(dfpen).select("features", "label")

In [6]:
penlpoints.show()

22/02/24 00:34:53 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+-----+
|            features|label|
+--------------------+-----+
|[47.0,100.0,27.0,...|  8.0|
|[0.0,89.0,27.0,10...|  2.0|
|[0.0,57.0,31.0,68...|  1.0|
|[0.0,100.0,7.0,92...|  4.0|
|[0.0,67.0,49.0,83...|  1.0|
|[100.0,100.0,88.0...|  6.0|
|[0.0,100.0,3.0,72...|  4.0|
|[0.0,39.0,2.0,62....|  0.0|
|[13.0,89.0,12.0,5...|  5.0|
|[74.0,87.0,31.0,1...|  9.0|
|[48.0,96.0,62.0,6...|  8.0|
|[100.0,100.0,72.0...|  5.0|
|[91.0,74.0,54.0,1...|  9.0|
|[0.0,85.0,38.0,10...|  7.0|
|[35.0,76.0,57.0,1...|  3.0|
|[50.0,84.0,66.0,1...|  3.0|
|[99.0,80.0,63.0,1...|  9.0|
|[24.0,66.0,43.0,1...|  2.0|
|[0.0,73.0,19.0,99...|  2.0|
|[12.0,77.0,20.0,6...|  5.0|
+--------------------+-----+
only showing top 20 rows



## Create Training and Test data.

In [7]:
# Create Training and Test data.
pendtsets = penlpoints.randomSplit([0.8, 0.2], 1)
pendttrain = pendtsets[0].cache()
pendtvalid = pendtsets[1].cache()

## Train the decision tree model

In [8]:
# Train the data.
from pyspark.ml.classification import DecisionTreeClassifier
# Paramenters
# maxDepth : maximum tree depth (default : 5).
# maxBins : maximum number of bins when binning continuous features (default : 32).
# minInstancesPerNode : minimum number of dataset samples 
#     each branch needs to have after a split (default : 1).
# minInfoGain : minimum information gain for a split (default : 0).
dt = DecisionTreeClassifier(maxDepth=20, maxBins= 32, minInstancesPerNode=1, minInfoGain = 0)
dtmodel = dt.fit(pendttrain)

In [9]:
dtmodel.featureImportances

SparseVector(16, {0: 0.0892, 1: 0.062, 2: 0.0248, 3: 0.1012, 4: 0.0993, 5: 0.0084, 6: 0.0292, 7: 0.0211, 8: 0.0227, 9: 0.1318, 10: 0.1269, 11: 0.0068, 12: 0.0403, 13: 0.0282, 14: 0.0572, 15: 0.1512})

In [10]:
print(dtmodel.toDebugString)

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_e5d5eaffec0e, depth=20, numNodes=589, numClasses=10, numFeatures=16
  If (feature 15 <= 51.5)
   If (feature 4 <= 41.5)
    If (feature 9 <= 16.5)
     If (feature 14 <= 63.5)
      Predict: 6.0
     Else (feature 14 > 63.5)
      If (feature 12 <= 55.5)
       If (feature 0 <= 38.5)
        Predict: 1.0
       Else (feature 0 > 38.5)
        Predict: 8.0
      Else (feature 12 > 55.5)
       If (feature 11 <= 9.5)
        If (feature 0 <= 20.5)
         Predict: 2.0
        Else (feature 0 > 20.5)
         Predict: 6.0
       Else (feature 11 > 9.5)
        If (feature 0 <= 0.5)
         Predict: 7.0
        Else (feature 0 > 0.5)
         Predict: 4.0
    Else (feature 9 > 16.5)
     If (feature 1 <= 99.5)
      If (feature 9 <= 62.5)
       If (feature 7 <= 18.5)
        Predict: 0.0
       Else (feature 7 > 18.5)
        If (feature 14 <= 16.5)
         If (feature 6 <= 51.5)
          If (feature 9 <= 26.5)
           Pre

## Test the model

In [11]:
#Test data.
dtpredicts = dtmodel.transform(pendtvalid)

In [12]:
dtpredicts.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[0.0,0.0,41.0,16....|  9.0|[0.0,0.0,0.0,0.0,...|[0.0,0.0,0.0,0.0,...|       9.0|
|[0.0,20.0,47.0,42...|  1.0|[0.0,378.0,0.0,0....|[0.0,1.0,0.0,0.0,...|       1.0|
|[0.0,22.0,36.0,47...|  1.0|[0.0,378.0,0.0,0....|[0.0,1.0,0.0,0.0,...|       1.0|
|[0.0,23.0,63.0,46...|  8.0|[0.0,0.0,0.0,0.0,...|[0.0,0.0,0.0,0.0,...|       8.0|
|[0.0,26.0,57.0,56...|  8.0|[0.0,0.0,0.0,0.0,...|[0.0,0.0,0.0,0.0,...|       8.0|
|[0.0,39.0,2.0,62....|  0.0|[1.0,0.0,0.0,0.0,...|[1.0,0.0,0.0,0.0,...|       0.0|
|[0.0,39.0,42.0,52...|  1.0|[0.0,378.0,0.0,0....|[0.0,1.0,0.0,0.0,...|       1.0|
|[0.0,40.0,29.0,56...|  1.0|[0.0,378.0,0.0,0....|[0.0,1.0,0.0,0.0,...|       1.0|
|[0.0,43.0,26.0,65...|  1.0|[0.0,378.0,0.0,0....|[0.0,1.0,0.0,0.0,...|       1.0|
|[0.0,43.0,35.0,

## Evaluate the model
available metrics : https://spark.apache.org/docs/latest/mllib-evaluation-metrics.html

In [13]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
# expects two input columns: prediction and label.

# f1|accuracy(defulat)|weightedPrecision|weightedRecall|weightedTruePositiveRate| 
# weightedFalsePositiveRate|weightedFMeasure|truePositiveRateByLabel| 
# falsePositiveRateByLabel|precisionByLabel|recallByLabel|fMeasureByLabel| 
# logLoss|hammingLoss
metric_name = "f1"

metrics = MulticlassClassificationEvaluator()\
                .setLabelCol("label")\
                .setPredictionCol("prediction")
metrics.setMetricName(metric_name) 

metrics.evaluate(dtpredicts)

0.9585890253909967

## N-fold validation
cross-validation : https://spark.apache.org/docs/latest/ml-tuning.html#cross-validation


In [14]:
# n-fold validation and the results.
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator()\
                .setLabelCol("label")\
                .setPredictionCol("prediction")
#ParamGridBuilder() – combinations of parameters and their values.
dt = DecisionTreeClassifier()
paramGrid = ParamGridBuilder().addGrid(dt.maxDepth, [5,10,15,20,25,30]).build()

cv = CrossValidator(estimator=dt, 
                    evaluator=evaluator, 
                    numFolds=5, 
                    estimatorParamMaps=paramGrid)

cvmodel = cv.fit(pendttrain)
dtpredicts = cvmodel.bestModel.transform(pendtvalid)

print("Best Max Depth : %s" % cvmodel.bestModel.getMaxDepth)
print("Accuracy : %s" % evaluator.evaluate(dtpredicts))

Best Max Depth : <bound method _DecisionTreeParams.getMaxDepth of DecisionTreeClassificationModel: uid=DecisionTreeClassifier_58c1a0a47697, depth=15, numNodes=569, numClasses=10, numFeatures=16>
Accuracy : 0.9585890253909967


In [15]:
from pyspark.mllib.evaluation import MulticlassMetrics

#prediction and label
prediction_label = dtpredicts.select("prediction", "label").rdd

metrics = MulticlassMetrics(prediction_label)

confusionMetrics = metrics.confusionMatrix()

print("Confusion Metrics = \n%s" % confusionMetrics)

/Users/dwoodbridge/opt/anaconda3/envs/DistributedComputing/lib/python3.7/site-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


Confusion Metrics = 
DenseMatrix([[223.,   0.,   0.,   0.,   0.,   0.,   1.,   0.,   4.,   0.],
             [  0., 197.,  12.,   3.,   0.,   0.,   2.,   0.,   0.,   0.],
             [  0.,   4., 200.,   0.,   1.,   0.,   1.,   1.,   0.,   0.],
             [  1.,   0.,   1., 185.,   0.,   0.,   0.,   1.,   0.,   3.],
             [  0.,   1.,   0.,   0., 216.,   0.,   1.,   0.,   0.,   2.],
             [  0.,   0.,   0.,   3.,   1., 176.,   2.,   2.,   2.,   9.],
             [  0.,   1.,   0.,   0.,   3.,   1., 192.,   1.,   0.,   0.],
             [  0.,   1.,   3.,   1.,   0.,   2.,   0., 189.,   0.,   0.],
             [  4.,   0.,   0.,   0.,   0.,   0.,   0.,   1., 169.,   0.],
             [  0.,   0.,   0.,   3.,   0.,   3.,   0.,   1.,   0., 178.]])


In [16]:
ss.stop()